# Optimal CHSH Violation on Quantum Hardware

This notebook demonstrates the performance of open-access quantum computers when given the task of implementing the strategy for optimal violation of the CHSH inequality.

In [9]:
import pennylane as qml
from pennylane import numpy as np

from context import QNetOptimizer as QNopt

In [2]:
from qiskit import IBMQ

# For details regarding integration between PennyLane and IMB Q,
# see https://pennylaneqiskit.readthedocs.io/en/latest/devices/ibmq.html#accounts-and-tokens
provider = IBMQ.load_account()

## Setup

The ansatz is exceedingly simple where a static Bell state is prepared and local qubit rotations about the $y$-axis are performed before computational basis measurement.

In [3]:
prep_nodes = [
    QNopt.PrepareNode(1, [0,1], QNopt.ghz_state, 0)
]
meas_nodes = [
    QNopt.MeasureNode(2, 2, [0], QNopt.local_RY, 1),
    QNopt.MeasureNode(2, 2, [1], QNopt.local_RY, 1)
]

The optimal settings are only needed for measurements.
In this case, the first measurement node (Alice) measures in either $\sigma_z$ or $\sigma_x$ bases and the second measurement node (Bob) applies measurements in orthogonal bases rotated off of the $x$ and $z$-bases.

Note: The optimal CHSH score is $2\sqrt{2} \approx 2.82$ while the CHSH cost function multiplies this score by $-1$.

In [4]:
opt_settings = [
    [np.array([[]])], # there are no preparation settings
    [np.array([[0],[np.pi/2]]), np.array([[np.pi/4],[-np.pi/4]])], # measurement settings
]

## Device: `default.qubit`

Local execution to demonstrate the optimal CHSH score and overhead of waiting in the queue for quantum hardware.

In [5]:
%%time

local_chsh_ansatz = QNopt.NetworkAnsatz(prep_nodes, meas_nodes)
parallel_chsh_cost = QNopt.chsh_inequality_cost(local_chsh_ansatz, parallel=True)

parallel_chsh_cost(opt_settings)

CPU times: user 138 ms, sys: 9.81 ms, total: 147 ms
Wall time: 157 ms


-2.828427124746189

## Device: `ibmq_belem`

A 5-qubit Falcon r4T processor with a quantum volume of 16.

In [6]:
%%time

dev_ibm_belem = {
    "name" : "qiskit.ibmq",
    "shots" : 4000,
    "backend" : "ibmq_belem",
    "provider" : provider
}

belem_chsh_ansatz = QNopt.NetworkAnsatz(prep_nodes, meas_nodes, dev_kwargs = dev_ibm_belem)
parallel_chsh_cost = QNopt.chsh_inequality_cost(belem_chsh_ansatz, parallel=True)

parallel_chsh_cost(opt_settings)

CPU times: user 1.16 s, sys: 131 ms, total: 1.29 s
Wall time: 40.5 s


-2.3550000000000004

## Device: `ibmq_manila`

A 5-qubit Falcon r5.11L processer with quantum volume of 32.

In [7]:
%%time

dev_ibm_manila = {
    "name" : "qiskit.ibmq",
    "shots" : 4000,
    "backend" : "ibmq_manila",
    "provider" : provider
}

manila_chsh_ansatz = QNopt.NetworkAnsatz(prep_nodes, meas_nodes, dev_kwargs = dev_ibm_manila)
parallel_chsh_cost = QNopt.chsh_inequality_cost(manila_chsh_ansatz, parallel=True)

parallel_chsh_cost(opt_settings)

CPU times: user 454 ms, sys: 42.9 ms, total: 497 ms
Wall time: 1min 27s


-2.418

## Device: `ibmq_santiago`

A 5-qubit Falcon r4L processor with quantum volume of 32.

In [8]:
%%time

dev_ibm_santiago = {
    "name" : "qiskit.ibmq",
    "shots" : 4000,
    "backend" : "ibmq_santiago",
    "provider" : provider
}

santiago_chsh_ansatz = QNopt.NetworkAnsatz(prep_nodes, meas_nodes, dev_kwargs = dev_ibm_santiago)
parallel_chsh_cost = QNopt.chsh_inequality_cost(santiago_chsh_ansatz, parallel=True)

parallel_chsh_cost(opt_settings)

CPU times: user 699 ms, sys: 93.7 ms, total: 792 ms
Wall time: 14min 18s


-2.4225000000000003

## Conclusion

Open-access devices at IBM are comparable in their ability to faithfully execute small simple circuits.
The CHSH score and and computation times are expected to deviate when run again and are therefore not reproducible.
It is likely that VQO will lead to greater CHSH violations as the theoretical optimal strategy may not be optimal for the noise in quantum processors. 